# Python crawler with taversal in Nested loop
# 이중 반복문을 활용한 파이썬 순회 크롤러 

In [2]:
# crwaling library import
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

# 코드 진행 지연을 위한 time 임포트
import time

# 2022-07 이후 selenium 업데이트로 인한 XPATH 추적 시 사용하는 임포트
from selenium.webdriver.common.by import By

# file io
import codecs

## 1. `bs4.element` 내부 item 접근

In [32]:
my_driver = webdriver.Chrome()

# 알라딘 베스트셀러 사이트 접속
my_driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BranchType=1&start=we")

# ip block 방지
time.sleep(10)

# 소스 얻기
my_source = my_driver.page_source

# parsing(page 전체)
my_parsed_source = BeautifulSoup(my_source, "html.parser")

my_driver.close()

### 1.1. web page element 조사 결과 > `"li"` tag에 책 정보(가격) 존재 > extract data by `li` tag

In [33]:
li_tag_data = my_parsed_source.find_all("li")

print(li_tag_data)

[<li><a href="https://www.aladin.co.kr/home/welcome.aspx" title="HOME"><img alt="HOME" src="//image.aladin.co.kr/img/header/2011/global/global_set1_m01.gif"/></a></li>, <li class="set2se"></li>, <li id="#head_book_layer"><a href="https://www.aladin.co.kr/home/wbookmain.aspx" title="국내도서"><img alt="국내도서" src="//image.aladin.co.kr/img/header/2011/global/global_set2_m01_on.gif"/></a>
<div style="position:relative;z-index:99999;">
<div class="hdr" id="head_book_layer" style="width:836px;">
<div style="position:relative;float:left">
<table border="0" cellpadding="5" cellspacing="4" style="margin:5px 0 0 10px">
<tbody><tr>
<td valign="top" width="114"> <table>
<tbody><tr>
<td><a class="gr03" href="https://www.aladin.co.kr/shop/wbrowse.aspx?CID=13789">유아</a></td>
</tr>
<tr>
<td><a class="gr03" href="https://www.aladin.co.kr/shop/wbrowse.aspx?CID=1108"><strong>어린이</strong></a></td>
</tr>
<tr>
<td><a class="gr03" href="https://www.aladin.co.kr/shop/wbrowse.aspx?CID=17195">전집/중고전집</a></td>
</tr>

- 정확한 데이터 추출 실패

<br>

### 1.2. 책 정보를 감싸고 있는 `"div"` tag and `"ss_book_box"` class로 data extraction

In [34]:
div_book_box = my_parsed_source.find_all("div", class_="ss_book_box")

print(div_book_box)
print(len(div_book_box))
print(type(div_book_box))

[<div class="ss_book_box" itemid="301692224">
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tbody><tr>
<td align="left" style="padding-right:5px;" valign="top" width="25">
<table border="0" cellpadding="0" cellspacing="0" width="18">
<tbody><tr><td><div style="text-align: center;">1.</div></td></tr> <tr><td><div style="text-align: center;"><input name="chkCart.8959897094" type="checkbox"/></div></td></tr> </tbody></table>
</td>
<td align="left" valign="top" width="170">
<table border="0" cellpadding="0" cellspacing="0" width="150">
<tbody><tr><td style=""><div style="padding: 0px 0px 5px 0px; text-align: center;"><table align="center" border="0" cellpadding="0" cellspacing="0" style="margin-top:5px;" width="100%"><tbody><tr><td style="font-size:11px;color:#000000;padding:2px 2px 0 2px;line-height:14px !important;text-align:center">
<span style="font-size:11px;color:#000000;padding:0 2px 0 2px">종합</span>Top10<font color="#fb6788" style="margin-left:3px;"><strong>2주</s

- `div_book_box` data == `bs4.element.ResultSet` object > `find_all()` method 없음

### 1.3. 내부 item 접근 방법

1. indexing으로 `bs4.element.ResultSet` 접근

2. `bs4.element.ResultSet[i]` > `find_all()` method > tag, class 기준으로 data 걸러내기

3. 걸러 낸 data > for loop 돌면서 item을 str으로 형변환: `item.text`

<br>

1. indexing으로 접근

In [35]:
print(div_book_box[0])

print(type(div_book_box[0]))

<div class="ss_book_box" itemid="301692224">
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tbody><tr>
<td align="left" style="padding-right:5px;" valign="top" width="25">
<table border="0" cellpadding="0" cellspacing="0" width="18">
<tbody><tr><td><div style="text-align: center;">1.</div></td></tr> <tr><td><div style="text-align: center;"><input name="chkCart.8959897094" type="checkbox"/></div></td></tr> </tbody></table>
</td>
<td align="left" valign="top" width="170">
<table border="0" cellpadding="0" cellspacing="0" width="150">
<tbody><tr><td style=""><div style="padding: 0px 0px 5px 0px; text-align: center;"><table align="center" border="0" cellpadding="0" cellspacing="0" style="margin-top:5px;" width="100%"><tbody><tr><td style="font-size:11px;color:#000000;padding:2px 2px 0 2px;line-height:14px !important;text-align:center">
<span style="font-size:11px;color:#000000;padding:0 2px 0 2px">종합</span>Top10<font color="#fb6788" style="margin-left:3px;"><strong>2주</st

- `bs4.element.Tag[0]`: 1위 책에 대한 정보 담겨있음

- type == `bs4.element.Tag` > `find_all()` method 사용 가능

<br>
2. `find_all()` method > "li" tag로 data 걸러내기

3. extracted data `item` > `item.text` > string으로 형 변환

In [36]:
first_li_book = div_book_box[0].find_all("li")

for item in first_li_book:
    print(item.text)

[토끼 골드머그, 스마트톡(이벤트 대상 도서 포함 국내도서 2만 원 이상)]
[국내도서] 트렌드 코리아 2023 
김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린 (지은이) | 미래의창 | 2022년 10월
19,000원 → 17,100원 (10%할인),  마일리지 950원 (5% 적립)
 (12) | 세일즈포인트 : 155,300
양탄자배송 내일 아침 7시 출근전 배송(중구 서소문로 89-31) 
이 책의 전자책 :  13,300원 
                                                
보관함
마이리스트


- `div_book_box[index N == N+1순위 책].find_all("li")` > for loop `bs4.element.Tag` \[index 1 ~ 3\]:  item.text > 책 정보 출력 가능으로 추측됨    
\>index N에 다른 숫자 넣어서 확인

In [37]:
print(first_li_book[1].text)
print(first_li_book[2].text)
print(first_li_book[3].text)

[국내도서] 트렌드 코리아 2023 
김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린 (지은이) | 미래의창 | 2022년 10월
19,000원 → 17,100원 (10%할인),  마일리지 950원 (5% 적립)


- `div_book_box[index N == N+1순위 책].find_all("li")` > for loop index 1 ~ 3: item.text > 책 정보 출력 가능으로 추측됨 > index N에 다른 숫자 넣어서 확인

In [38]:
for i in range(10, 14): # N = 10 ~ 13 > 10위 ~ 13위 책
    this_li_book = div_book_box[i].find_all("li")
    
    for j in range(1, 4): #index 1, 2, 3
        print(this_li_book[j].text)
        
    print()

[국내도서] 총 균 쇠 (리커버 특별판) 
재레드 다이아몬드 (지은이), 김진준 (옮긴이) | 문학사상 | 2022년 10월
28,000원 → 25,200원 (10%할인),  마일리지 1,400원 (5% 적립)

[국내도서] 올리버쌤의 미국식 아이 영어 습관 365 
올리버 샨 그랜트 (지은이), 정다운 (그림) | 브라이트(다산북스) | 2022년 10월
23,000원 → 20,700원 (10%할인),  마일리지 1,150원 (5% 적립)

[국내도서] 가녀장의 시대  
이슬아 (지은이) | 이야기장수 | 2022년 10월
15,000원 → 13,500원 (10%할인),  마일리지 750원 (5% 적립)

[국내도서] 불편한 편의점 (40만부 기념 벚꽃 에디션) 
김호연 (지은이) | 나무옆의자 | 2021년 4월
14,000원 → 12,600원 (10%할인),  마일리지 700원 (5% 적립)



- **규칙성 확인 완료** > _data crawler_ 사용 가능!

<br>

- - -

## 2. parsed_source를 for loop로 한 권씩 책 정보 출력   

- text.split() > 원하는 정보 골라서 출력 가능

### 2.1. 한 webpage의 parsed_source 

- source = `parsed_source.find_all()` > `for loop` source\[indexing\] (== item),   
`item.find_all()` > 원하는 data indexing

In [46]:
div_book_box = my_parsed_source.find_all("div", class_="ss_book_box")

i = 1
for this_div_book in div_book_box:
    this_li_book = this_div_book.find_all("li")
    
    print(i)
    
    this_title = this_li_book[1].text.split("]")[1]
    this_author = this_li_book[2].text.split(" (")[0]
    this_price = this_li_book[3].text.split("원 →")[0].replace(",", "，")
    
    print(this_title, this_author, this_price)
    i += 1

1
 트렌드 코리아 2023  김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린 19，000
2
 아버지의 해방일지   정지아 15，000
3
 이토록 평범한 미래   김연수 14，000
4
 단순한 열정 (무선) ㅣ 문학동네 세계문학전집 99  아니 에르노 10，000
5
 세상에서 가장 쉬운 본질육아  지나영 18，800
6
 헤어질 결심 스토리보드북   이윤호, 박찬욱 33，000
7
 불편한 편의점 2  김호연 14，000
8
 역행자  자청 17，500
9
 물고기는 존재하지 않는다 (리커버 특별판, 양장)  룰루 밀러 17，000
10
 하얼빈   김훈 16，000
11
 총 균 쇠 (리커버 특별판)  재레드 다이아몬드 28，000
12
 올리버쌤의 미국식 아이 영어 습관 365  올리버 샨 그랜트 23，000
13
 가녀장의 시대   이슬아 15，000
14
 불편한 편의점 (40만부 기념 벚꽃 에디션)  김호연 14，000
15
 작은 땅의 야수들  김주혜 18，000
16
 세월 ㅣ 아니 에르노 컬렉션   아니 에르노 15，500
17
 아름다운 초저녁달 4  야마모리 미카 6，000
18
 파친코 1  이민진 15，800
19
 흔한남매의 흔한 호기심 7 ㅣ 흔한남매   안치현 14，000
20
 파친코 2  이민진 15，800
21
 나는 오래된 거리처럼 너를 사랑하고 ㅣ 문학과지성 시인선 572   진은영 12，000
22
 2023 써니 행정법총론 기출문제집 - 전2권  박준철 42，000
23
 쇼타 형아 2  나카야마 미유키 6，500
24
 이 책은 돈 버는 법에 관한 이야기  고명환 16，800
25
 사람을 얻는 지혜  발타자르 그라시안 13，800
26
 마지막 이야기 전달자   도나 바르바 이게라 18，000
27
 잘될 수밖에 없는 너에게  최서영 16，000
28
 2022 김승옥문학상 수상작품집   편혜영, 김연수, 김애란, 정한아, 문지혁, 백수린 10，000
29
 그 길로 갈 

IndexError: list index out of range

- `IndexError: list index out of range` 원인

\> `[50번].find_all('li')[1]` != title, `50번.find_all('li')[0]` == title

\> 다른 책에는 index 0에 증정품 item 있음

\> 50번 째 책에는 증정품 없이 바로 title item > index 0에 title item 존재

### 2.2. indexing 결과와 원하는 item이 다를 때 처리 방법

In [48]:
# 구글웹드라이버 사용
my_driver = webdriver.Chrome()

# 알라딘 베스트셀러 사이트 접속
my_driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page=1&cnt=1000&SortOrder=1")

# ip block 방지
time.sleep(10)

# 소스 얻기
my_source = my_driver.page_source

# parsing(page 전체)
my_parsed_source = BeautifulSoup(my_source, "html.parser")

my_driver.close()

- str로 형변환 된 source code > `[N번].find_all('li')[0]`: 첫 번째 index item    
\> 만약 ']'로 끝나면 > title 아니고 증정품 > `index += 1` 수행

In [49]:
div_book_box = my_parsed_source.find_all("div", class_="ss_book_box")

for this_div_book in div_book_box:
    this_li_book = this_div_book.find_all("li")
    this_td_book = this_div_book.find_all("td")

    first_line = this_li_book[0].text
    # 첫째 line이 ']'로 끝나면 (증정품) > index 밀기
    nIndex = 0
    if (first_line[len(first_line) - 1] == ']'):
        nIndex = 1
    
    this_rank = this_td_book[1].text
    this_title= this_li_book[nIndex].text.split("]")[1]
    this_author = this_li_book[nIndex + 1].text.split(" (")[0]
    this_price = this_li_book[nIndex + 2].text.split("원 →")[0].replace(",", "，")
    
    print(this_rank, this_title, this_author, this_price)

1.  트렌드 코리아 2023  김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린 19，000
2.  아버지의 해방일지   정지아 15，000
3.  이토록 평범한 미래   김연수 14，000
4.  단순한 열정 (무선) ㅣ 문학동네 세계문학전집 99  아니 에르노 10，000
5.  세상에서 가장 쉬운 본질육아  지나영 18，800
6.  헤어질 결심 스토리보드북   이윤호, 박찬욱 33，000
7.  불편한 편의점 2  김호연 14，000
8.  역행자  자청 17，500
9.  물고기는 존재하지 않는다 (리커버 특별판, 양장)  룰루 밀러 17，000
10.  하얼빈   김훈 16，000
11.  총 균 쇠 (리커버 특별판)  재레드 다이아몬드 28，000
12.  올리버쌤의 미국식 아이 영어 습관 365  올리버 샨 그랜트 23，000
13.  가녀장의 시대   이슬아 15，000
14.  불편한 편의점 (40만부 기념 벚꽃 에디션)  김호연 14，000
15.  작은 땅의 야수들  김주혜 18，000
16.  세월 ㅣ 아니 에르노 컬렉션   아니 에르노 15，500
17.  아름다운 초저녁달 4  야마모리 미카 6，000
18.  파친코 1  이민진 15，800
19.  흔한남매의 흔한 호기심 7 ㅣ 흔한남매   안치현 14，000
20.  파친코 2  이민진 15，800
21.  나는 오래된 거리처럼 너를 사랑하고 ㅣ 문학과지성 시인선 572   진은영 12，000
22.  2023 써니 행정법총론 기출문제집 - 전2권  박준철 42，000
23.  쇼타 형아 2  나카야마 미유키 6，500
24.  이 책은 돈 버는 법에 관한 이야기  고명환 16，800
25.  사람을 얻는 지혜  발타자르 그라시안 13，800
26.  마지막 이야기 전달자   도나 바르바 이게라 18，000
27.  잘될 수밖에 없는 너에게  최서영 16，000
28.  2022 김승옥문학상 수상작품집   편혜영, 김연수, 김애란, 정한아,

- - -

## 3. 순회 크롤링 in 2중 for loop (crawler with traversal in Nested loop)

In [50]:
# 구글웹드라이버 사용
my_driver = webdriver.Chrome()

# 리스트에 item append
rank_list = list()
title_list = list()
author_list = list()
price_list = list()

# 첫 번째 for loop: 사이트 traversal
for i in range(1, 21):
    # 알라딘 베스트셀러 사이트 n페이지 접속
    my_driver.get("https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page=%s&cnt=1000&SortOrder=1" %i)

    # ip block 방지
    time.sleep(10)

    # 소스 얻기
    my_source = my_driver.page_source

    # parsing(page 전체)
    my_parsed_source = BeautifulSoup(my_source, "html.parser")
    
    div_book_box = my_parsed_source.find_all("div", class_="ss_book_box")
    
    # 두 번째 for loop: 페이지 내의 class ss_book_box item traversal 
    for this_div_book in div_book_box:
        this_li_book = this_div_book.find_all("li")
        
        # rank item tag "td"
        this_td_book = this_div_book.find_all("td")
        
        first_line = this_li_book[0].text
        # 첫째 line이 ']'로 끝나면 (증정품) > index 밀기
        nIndex = 0
        if (first_line[len(first_line) - 1] == ']'):
            nIndex = 1
    
        this_rank = this_td_book[1].text
        
        this_title= this_li_book[nIndex].text.split("]")[1]
        this_author = this_li_book[nIndex + 1].text.split(" (")[0]
        this_price = this_li_book[nIndex + 2].text.split(" →")[0].replace("원","").replace(",", "，")
    
        rank_list.append(this_rank)
        title_list.append(this_title)
        author_list.append(this_author)
        price_list.append(this_price)
    
        print(this_rank, this_title, this_author, this_price)

my_driver.close()

1.  트렌드 코리아 2023  김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린 19，000
2.  아버지의 해방일지   정지아 15，000
3.  이토록 평범한 미래   김연수 14，000
4.  단순한 열정 (무선) ㅣ 문학동네 세계문학전집 99  아니 에르노 10，000
5.  세상에서 가장 쉬운 본질육아  지나영 18，800
6.  헤어질 결심 스토리보드북   이윤호, 박찬욱 33，000
7.  불편한 편의점 2  김호연 14，000
8.  역행자  자청 17，500
9.  물고기는 존재하지 않는다 (리커버 특별판, 양장)  룰루 밀러 17，000
10.  하얼빈   김훈 16，000
11.  총 균 쇠 (리커버 특별판)  재레드 다이아몬드 28，000
12.  올리버쌤의 미국식 아이 영어 습관 365  올리버 샨 그랜트 23，000
13.  가녀장의 시대   이슬아 15，000
14.  불편한 편의점 (40만부 기념 벚꽃 에디션)  김호연 14，000
15.  작은 땅의 야수들  김주혜 18，000
16.  세월 ㅣ 아니 에르노 컬렉션   아니 에르노 15，500
17.  아름다운 초저녁달 4  야마모리 미카 6，000
18.  파친코 1  이민진 15，800
19.  흔한남매의 흔한 호기심 7 ㅣ 흔한남매   안치현 14，000
20.  파친코 2  이민진 15，800
21.  나는 오래된 거리처럼 너를 사랑하고 ㅣ 문학과지성 시인선 572   진은영 12，000
22.  2023 써니 행정법총론 기출문제집 - 전2권  박준철 42，000
23.  쇼타 형아 2  나카야마 미유키 6，500
24.  이 책은 돈 버는 법에 관한 이야기  고명환 16，800
25.  사람을 얻는 지혜  발타자르 그라시안 13，800
26.  마지막 이야기 전달자   도나 바르바 이게라 18，000
27.  잘될 수밖에 없는 너에게  최서영 16，000
28.  2022 김승옥문학상 수상작품집   편혜영, 김연수, 김애란, 정한아,

251.  참을 수 없는 존재의 가벼움  밀란 쿤데라 15，000
252.  백년운동   정선근 19，500
253.  송사무장의 부동산 경매의 기술  송희창 16，000
254.  도올주역강해  김용옥 39，000
255.  딜레마 사전 ㅣ 작가들을 위한 사전 시리즈   안젤라 애커만, 베카 푸글리시 22，000
256.  예쁘게 말하는 네가 좋다  김범준 15，800
257.  3년 후 부의 흐름이 보이는 경제지표 정독법  김영익 18，500
258.  계속 가보겠습니다  임은정 18，000
259.  순례 주택 ㅣ 블루픽션 (비룡소 청소년 문학선) 81  유은실 13，000
260.  죽음의 수용소에서   빅터 프랭클 9，800
261.  존재의 세 가지 거짓말 (양장)  아고타 크리스토프 22，000
262.  백 번 산 고양이 백꼬선생 1 ㅣ 우리학교 상상 도서관   정연철 13，000
263.  카사노바 호텔  아니 에르노 13，500
264.  내 멋대로 초능력 뽑기 ㅣ 내 멋대로 뽑기   최은옥 12，500
265.  내가 틀릴 수도 있습니다  비욘 나티코 린데블라드 16，000
266.  브라질에 비가 내리면 스타벅스 주식을 사라  피터 나바로 18，000
267.  뇌, 욕망의 비밀을 풀다   한스-게오르크 호이젤 18，000
268.  눈감지 마라 ㅣ 마음산책 짧은 소설    이기호 15，000
269.  수상한 지하실 ㅣ 북멘토 가치동화 48  박현숙 13，000
270.  클라우드 머니  브렛 스콧 19，000
271.  해커스 토익 RC 리딩 READING ㅣ 해커스 토익 기본서   David Cho 18，800
272.  ETS 토익 기출 VOCA 출제기관 공식수험서  ETS 12，900
273.  명탐정 코난 101   아오야마 고쇼 5，500
274.  마음이 흐르는 대로  지나영 16，000
275.  그리스 로마 신화 30 : 기나긴 모험 그 끝의 이야기 ㅣ 만화로 읽는 초등 인문학   박시연 15，000
276.  괜찮아, 

501.  나는 주식 대신 달러를 산다  박성현 16，000
502.  현명한 투자자  벤저민 그레이엄 23，000
503.  열혈강호 86  전극진 5，500
504.  노르웨이의 숲  무라카미 하루키 15，000
505.  Go Go 카카오프렌즈 25 : 네덜란드 ㅣ Go Go 카카오프렌즈 25  김미영 14，800
506.  도쿄 에일리언즈 2  NAOE 5，000
507.  아름다운 초저녁달 3  야마모리 미카 6，000
508.  에어리얼   실비아 플라스 17，000
509.  우리 반 목소리 작은 애 ㅣ 동화 쫌 읽는 어린이   김수현 12，000
510.  역사적 그리스도와 신학적 예수 ㅣ 비아 시선들   데일 C. 앨리슨 14，000
511.  빨리 하고 싶은 두 사람 4  쿠사카 아키 6，000
512.  사실은 내가 가장 듣고 싶던 말  따듯한 목소리 현준 15，800
513.  어린이라는 세계   김소영 15，000
514.  둥실이네 떡집 ㅣ 난 책읽기가 좋아   김리리 11，000
515.  도쿄 에일리언즈 1  NAOE 5，000
516.  스파이 패밀리 3  엔도 타츠야 6，000
517.  다락방의 미친 여자   샌드라 길버트, 수전 구바 55，000
518.  외계에서 온 펀자이씨 ㅣ 펀자이씨툰 2   엄유진 16，000
519.  똑게육아  로리(김준희) 22，200
520.  홍길동전 : 춤추는 소매 바람을 따라 휘날리니 ㅣ 국어시간에 고전읽기 (나라말) 3  류수열 11，000
521.  아홉살 인생 ㅣ 현북스 소설 1  위기철 12，000
522.  마틴 에덴 1  잭 런던 18，800
523.  아틀라스 한국사 ㅣ 아틀라스 역사 시리즈 1  한국교원대학교 역사교육과 교수진 36，000
524.  현대 철학의 최전선  나카마사 마사키 18，000
525.  그들의 말 혹은 침묵  아니 에르노 14，000
526.  ETS 토익 정기시험 기출문제집 1000 Vol. 2 Reading (리딩) ㅣ ETS 토익 정기시험 기출문

701.  아몬드 (100만 부 기념 특별판, 양장)  손원평 12，000
702.  혼자 공부하는 파이썬 ㅣ 혼자 공부하는 시리즈   윤인성 22，000
703.  시지프 신화 ㅣ 민음사 세계문학전집 343  알베르 카뮈 12，000
704.  공정 이후의 세계  김정희원 17，000
705.  예민함이라는 선물  이미 로 17，000
706.  가벼운 마음  크리스티앙 보뱅 14，000
707.  전지적 독자 시점 Part 1 01   싱숑 13，500
708.  사카모토 데이즈 6  스즈키 유우토 6，000
709.  악당의 아빠를 꼬셔라 5 (캐릭터 책갈피 1종 + 클리어 스탠드 1종 + 체크리스트형 빅 메모지 + 포토카드 10종 + 아크릴 키링 + 일러스트 달력 + 박스 포함 한정판)  비아 35，000
710.  부족한 것이 없는 사람에게 왜 복음이 필요한가?  윌리엄 윌리몬 12，000
711.  리틀 라이프 1  한야 야나기하라 14，800
712.  운명을 바꾸는 부동산 투자 수업 (11만 부 기념 바이블 에디션) ㅣ 운명을 바꾸는 부동산 투자 수업   부동산읽어주는남자(정태익) 30，000
713.  2022 투자자산운용사 한권으로 끝내기 ver 8.0  유창호 34，000
714.  월간최신 취업에 강한 에듀윌 시사상식 2022.10  에듀윌 상식연구소 10，000
715.  처음 가는 마을 ㅣ 봄날의책 세계시인선 3  이바라기 노리코 11，000
716.  2023 퍼시픽 간호사 국시대비 문제집 9 : 보건의료법규 ㅣ 2023 퍼시픽 간호사 국시대비 문제집 9  퍼시픽북스 학술편찬국 21，000
717.  에브리바디  올리비아 랭 17，800
718.  쉽게 보는 난중일기 완역본  이순신 16，000
719.  영웅왕, 극한의 무를 위해 전생하다 7  하야켄 9，800
720.  정주영이 누구예요  이민우 20，000
721.  벌거벗은 한국사 : 사건편 ㅣ 벌거벗은 한국사   tvN〈벌거벗은 한국사〉제작팀 18，800
722.  갑자기 

901.  2023 선재국어 세트 (전2권 + 부록)  이선재 47，000
902.  슬램덩크 신장재편판 1  이노우에 타케히코 7，000
903.  설민석의 삼국지 대모험 8 ㅣ 설민석의 삼국지 대모험 8  단꿈아이 11，000
904.  게임하고 싶어! ㅣ 김영진 그림책 16  김영진 14，000
905.  나를 아프게 한 건 항상 나였다  이혜진 16，000
906.  전지적 독자 시점 Part 1 02   싱숑 13，500
907.  매일매일 행복해 ㅣ 피카 그림책 2  프란체스카 피로네 14，000
908.  전지적 독자 시점 Part 1 03   싱숑 13，500
909.  전지적 독자 시점 Part 1 04   싱숑 13，500
910.  도서관을 훔친 아이 ㅣ 그래요 책이 좋아요 3  알프레드 고메스 세르다 12，500
911.  가부키초 배드 트립  나기사 에이지 6，000
912.  짐승일기  김지승 16，000
913.  혼자 시작하는 사주명리 공부  김원 28，000
914.  프로가 되기 위한 작화 기술  오시야마 키요타카 15，000
915.  2022 해커스 NCS 지역농협 6급 통합 기본서 인적성 및 직무능력평가  해커스 취업교육연구소 25，900
916.  2023 권규호 공무원 국어 개기문 : 문법 ㅣ 2023 권규호 공무원 국어   권규호 19，000
917.  KBS 한국어능력시험 기출문제 18  KBS한국어진흥원 29，000
918.  까먹어도 될까요 ㅣ 첫 읽기책 16  유은실 10，000
919.  2023 김중규 여다나 압축 선행정학  김중규 25，000
920.  흉터 쿠키 ㅣ 현대문학 핀 시리즈 시인선 42   이혜미 9，000
921.  가르칠 수 없는 것을 가르치기 ㅣ 배우는 사람, 교사   이병곤 16，500
922.  가벼운 선물 ㅣ 민음의 시 301   조해주 12，000
923.  우리의 정원 ㅣ 사계절 1318 문고 137  김지현 12，000
924.  아인슈타인의 베일  안톤 차일링거 15，000


In [51]:
info_list = [rank_list, title_list, author_list, price_list]
for info in info_list:
    print(len(info))

999
999
999
999


- 1000위까지 있는데 len(list) == 999 ? > 중간에 **빠진 index check**

In [52]:
rank_check = []
    
for rank in rank_list:
    rank_check.append(int(rank.split(".")[0]))

for i in range(1, 1001):
    if not i in rank_check:
        print(i)

63


- 빠진 index == 63 > 실제 webpage에서 63th rank 없는 것 확인 완료 > data 출력

In [56]:
# .csv file로 출력
w = codecs.open("C:/Users/EthanJ/develop/Basic_Python_with_Data/crawling/crawler_with_traversal_in_Nested_loop.csv", encoding="utf-8-sig", mode="w")

this_line = str()

for i in range(len(rank_list)):
    this_line = str("%s,%s,%s,%s\n" %(rank_list[i].replace(",", "，"), title_list[i].replace(',', '，'),
                                      price_list[i].replace(',', '，'), author_list[i].replace(',', '，')))
    
    w.write(this_line)
    
w.close()

> crawler_with_traversal_in_Nested_loop.csv 캡쳐